In [1]:
import sqlite3
from sqlite3 import Error

In [2]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

In [3]:
def select_all_tasks(conn, query1):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    #query1 = """
        #SELECT *
        #FROM FACILITIES
        #"""
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)

In [4]:
def main(query1):
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn, query1)

Q1: Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do.

In [5]:
query1 = """
        SELECT name
        FROM FACILITIES
        WHERE membercost>0
        """
main(query1)

2.6.0
2. Query all tasks
('Tennis Court 1',)
('Tennis Court 2',)
('Massage Room 1',)
('Massage Room 2',)
('Squash Court',)


Q2: How many facilities do not charge a fee to members?

In [6]:
query1 = """
        SELECT COUNT(*) 
        FROM FACILITIES
        WHERE membercost=0
        """
main(query1)

2.6.0
2. Query all tasks
(4,)


Q3: Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question.

In [7]:
query1 = """
        SELECT facid, name, membercost, monthlymaintenance
        FROM FACILITIES
        WHERE membercost>0
        AND membercost < monthlymaintenance*0.2
        """
main(query1)

2.6.0
2. Query all tasks
(0, 'Tennis Court 1', 5, 200)
(1, 'Tennis Court 2', 5, 200)
(4, 'Massage Room 1', 9.9, 3000)
(5, 'Massage Room 2', 9.9, 3000)
(6, 'Squash Court', 3.5, 80)


Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator.

In [8]:
query1 = """
        SELECT *
        FROM FACILITIES
        WHERE facid in (1,5)
        """
main(query1)

2.6.0
2. Query all tasks
(1, 'Tennis Court 2', 5, 25, 8000, 200)
(5, 'Massage Room 2', 9.9, 80, 4000, 3000)


Q5: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question.

In [9]:
query1 = """
        SELECT name, monthlymaintenance,
        CASE WHEN monthlymaintenance > 100 THEN 'expensive' ELSE 'cheap' END AS label
        FROM FACILITIES
        """
main(query1)

2.6.0
2. Query all tasks
('Tennis Court 1', 200, 'expensive')
('Tennis Court 2', 200, 'expensive')
('Badminton Court', 50, 'cheap')
('Table Tennis', 10, 'cheap')
('Massage Room 1', 3000, 'expensive')
('Massage Room 2', 3000, 'expensive')
('Squash Court', 80, 'cheap')
('Snooker Table', 15, 'cheap')
('Pool Table', 15, 'cheap')


Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution.

In [10]:
query1 = """
        SELECT firstname, surname
        FROM Members
        WHERE joindate = (
        SELECT MAX(joindate) 
        FROM Members)
        """
main(query1)

2.6.0
2. Query all tasks
('Darren', 'Smith')


Q7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name.

In [11]:
query1 = """ 
        SELECT sub.court, sub.firstname||' '||sub.surname AS name
        FROM 
        (SELECT Facilities.name AS court, Members.firstname AS firstname, Members.surname AS surname
        FROM Bookings
        INNER JOIN Facilities ON Bookings.facid = Facilities.facid
        AND Facilities.name LIKE  'Tennis Court%'
        INNER JOIN Members ON Bookings.memid = Members.memid)
        sub
        GROUP BY sub.court, sub.firstname, sub.surname
        ORDER BY name
        """
main(query1)

2.6.0
2. Query all tasks
('Tennis Court 1', 'Anne Baker')
('Tennis Court 2', 'Anne Baker')
('Tennis Court 1', 'Burton Tracy')
('Tennis Court 2', 'Burton Tracy')
('Tennis Court 1', 'Charles Owen')
('Tennis Court 2', 'Charles Owen')
('Tennis Court 2', 'Darren Smith')
('Tennis Court 1', 'David Farrell')
('Tennis Court 2', 'David Farrell')
('Tennis Court 1', 'David Jones')
('Tennis Court 2', 'David Jones')
('Tennis Court 1', 'David Pinker')
('Tennis Court 1', 'Douglas Jones')
('Tennis Court 1', 'Erica Crumpet')
('Tennis Court 1', 'Florence Bader')
('Tennis Court 2', 'Florence Bader')
('Tennis Court 1', 'GUEST GUEST')
('Tennis Court 2', 'GUEST GUEST')
('Tennis Court 1', 'Gerald Butters')
('Tennis Court 2', 'Gerald Butters')
('Tennis Court 2', 'Henrietta Rumney')
('Tennis Court 1', 'Jack Smith')
('Tennis Court 2', 'Jack Smith')
('Tennis Court 1', 'Janice Joplette')
('Tennis Court 2', 'Janice Joplette')
('Tennis Court 1', 'Jemima Farrell')
('Tennis Court 2', 'Jemima Farrell')
('Tennis Court 1

Q8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries.

In [12]:
query1 = """
        SELECT Facilities.name AS facility, Members.firstname||' '||Members.surname  AS name, 
        CASE WHEN Bookings.memid = 0
        THEN Facilities.guestcost * Bookings.slots
        ELSE Facilities.membercost * Bookings.slots
        END AS cost
        FROM Bookings
        INNER JOIN Facilities ON Bookings.facid = Facilities.facid
        AND Bookings.starttime LIKE '2012-09-14%'
        AND (((Bookings.memid =0) AND (Facilities.guestcost * Bookings.slots >30))
        OR ((Bookings.memid !=0) AND (Facilities.membercost * Bookings.slots >30)))
        INNER JOIN Members ON Bookings.memid = Members.memid
        ORDER BY cost DESC
        """
main(query1)

2.6.0
2. Query all tasks
('Massage Room 2', 'GUEST GUEST', 320)
('Massage Room 1', 'GUEST GUEST', 160)
('Massage Room 1', 'GUEST GUEST', 160)
('Massage Room 1', 'GUEST GUEST', 160)
('Tennis Court 2', 'GUEST GUEST', 150)
('Tennis Court 1', 'GUEST GUEST', 75)
('Tennis Court 1', 'GUEST GUEST', 75)
('Tennis Court 2', 'GUEST GUEST', 75)
('Squash Court', 'GUEST GUEST', 70.0)
('Massage Room 1', 'Jemima Farrell', 39.6)
('Squash Court', 'GUEST GUEST', 35.0)
('Squash Court', 'GUEST GUEST', 35.0)


Q9: This time, produce the same result as in Q8, but using a subquery.

In [13]:
query1 = """
        SELECT * 
        FROM (
        SELECT Facilities.name AS facility, Members.firstname||' '||Members.surname AS name, 
        CASE WHEN Bookings.memid = 0
        THEN Facilities.guestcost * Bookings.slots
        ELSE Facilities.membercost * Bookings.slots
        END AS cost
        FROM Bookings
        INNER JOIN Facilities ON Bookings.facid = Facilities.facid
        AND Bookings.starttime LIKE '2012-09-14%'
        INNER JOIN Members ON Bookings.memid = Members.memid
        )sub
        WHERE sub.cost >30
        ORDER BY sub.cost DESC
        """
main(query1)

2.6.0
2. Query all tasks
('Massage Room 2', 'GUEST GUEST', 320)
('Massage Room 1', 'GUEST GUEST', 160)
('Massage Room 1', 'GUEST GUEST', 160)
('Massage Room 1', 'GUEST GUEST', 160)
('Tennis Court 2', 'GUEST GUEST', 150)
('Tennis Court 1', 'GUEST GUEST', 75)
('Tennis Court 1', 'GUEST GUEST', 75)
('Tennis Court 2', 'GUEST GUEST', 75)
('Squash Court', 'GUEST GUEST', 70.0)
('Massage Room 1', 'Jemima Farrell', 39.6)
('Squash Court', 'GUEST GUEST', 35.0)
('Squash Court', 'GUEST GUEST', 35.0)


Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!

In [14]:
query1 = """
    SELECT * 
    FROM (
    SELECT sub.facility, SUM( sub.cost ) AS total_revenue
    FROM (
    SELECT Facilities.name AS facility, 
    CASE WHEN Bookings.memid =0
    THEN Facilities.guestcost * Bookings.slots
    ELSE Facilities.membercost * Bookings.slots
    END AS cost
    FROM Bookings
    INNER JOIN Facilities ON Bookings.facid = Facilities.facid
    INNER JOIN Members ON Bookings.memid = Members.memid
    )sub
    GROUP BY sub.facility
    )sub2
    WHERE sub2.total_revenue <1000
        """
main(query1)

2.6.0
2. Query all tasks
('Pool Table', 270)
('Snooker Table', 240)
('Table Tennis', 180)


Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

In [17]:
query1 = """ 
        SELECT m1.surname||' '||m1.firstname AS member, 
        m2.surname || " " || m2.firstname AS recommended_by
        FROM Members AS m1
        INNER JOIN Members AS m2
        ON m1.recommendedby = m2.memid
        WHERE m1.recommendedby IS NOT NULL AND m1.recommendedby <> 0
        ORDER BY member
        """
main(query1)

2.6.0
2. Query all tasks
('Bader Florence', 'Stibbons Ponder')
('Baker Anne', 'Stibbons Ponder')
('Baker Timothy', 'Farrell Jemima')
('Boothe Tim', 'Rownam Tim')
('Butters Gerald', 'Smith Darren')
('Coplin Joan', 'Baker Timothy')
('Crumpet Erica', 'Smith Tracy')
('Dare Nancy', 'Joplette Janice')
('Genting Matthew', 'Butters Gerald')
('Hunt John', 'Purview Millicent')
('Jones David', 'Joplette Janice')
('Jones Douglas', 'Jones David')
('Joplette Janice', 'Smith Darren')
('Mackenzie Anna', 'Smith Darren')
('Owen Charles', 'Smith Darren')
('Pinker David', 'Farrell Jemima')
('Purview Millicent', 'Smith Tracy')
('Rumney Henrietta', 'Genting Matthew')
('Sarwin Ramnaresh', 'Bader Florence')
('Smith Jack', 'Smith Darren')
('Stibbons Ponder', 'Tracy Burton')
('Worthington-Smyth Henry', 'Smith Tracy')


Q12: Find the facilities with their usage by member, but not guests

In [19]:
query1 = """ 
        SELECT f.name AS Facility,
        SUM(Slots) AS total_usage
        FROM Bookings as b
        LEFT JOIN Facilities as f
        ON f.facid = b.facid
        WHERE b.memid > 0
        GROUP BY Facility
        """
main(query1)

2.6.0
2. Query all tasks
('Badminton Court', 1086)
('Massage Room 1', 884)
('Massage Room 2', 54)
('Pool Table', 856)
('Snooker Table', 860)
('Squash Court', 418)
('Table Tennis', 794)
('Tennis Court 1', 957)
('Tennis Court 2', 882)


Q13: Find the facilities usage by month, but not guests

In [22]:
query1 = """ 
        SELECT f.name AS facility,
          strftime('%,m', starttime) AS month,
          SUM(slots) AS total_usage
        FROM Bookings as b
        LEFT JOIN Facilities as f
        ON f.facid = b.facid
        GROUP BY facility, month;
        """
main(query1)

2.6.0
2. Query all tasks
('Badminton Court', None, 1209)
('Massage Room 1', None, 1404)
('Massage Room 2', None, 228)
('Pool Table', None, 910)
('Snooker Table', None, 908)
('Squash Court', None, 1104)
('Table Tennis', None, 830)
('Tennis Court 1', None, 1320)
('Tennis Court 2', None, 1278)
